<a href="https://colab.research.google.com/github/nntu079/Basic-Pytorch/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [12]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('test test')

In [13]:
#Nx1x28x28 
# Hyperparameters
input_size=28
sequence_length=28
num_layers=2
hidden_size=256
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs=2

In [14]:
# Create RNN
class RNN(nn.Module):
  def __init__(self,input_size,hidden_size,num_layers,num_classes):
    super(RNN,self).__init__()
    self.hidden_size=hidden_size
    self.num_layers=num_layers
    self.rnn= nn.RNN(input_size,hidden_size,num_layers,batch_first=True)
    self.fc=nn.Linear(hidden_size*sequence_length,num_classes)

  def forward(self,x):
    h0=torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)
    
    out,_=self.rnn(x,h0)
    #print(out.shape) # [64, 28, 256]
    out=out.reshape(out.shape[0],-1);
    # out= out[:,-1:] => many to one
    #print(out.shape) # [64, 7168]
    out=self.fc(out)
    return out;

model = RNN(input_size,hidden_size,num_layers,num_classes);
x=torch.randn(batch_size,sequence_length,input_size);

print(x.shape)
print(model(x).shape)

torch.Size([64, 28, 28])
torch.Size([64, 10])


In [15]:
# Load Data
patch=r'/content/drive/MyDrive/Môn học/Pytorch/mnist';
train_dataset=datasets.MNIST(root=patch,train=True,transform=transforms.ToTensor(),download=True)
train_loader= DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset=datasets.MNIST(root=patch,train=False,transform=transforms.ToTensor(),download=True)
test_loader= DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

In [16]:
# Initialize Network
model= RNN(input_size,hidden_size,num_layers,num_classes).to(device)

In [17]:
# Loss and optimizer
criterion= nn.CrossEntropyLoss();
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [18]:
x=torch.randn(batch_size,sequence_length,input_size);
print(x.shape)
x=x.reshape(1,64,28,28)

x=x.squeeze(1)
print(x.shape)

torch.Size([64, 28, 28])
torch.Size([1, 64, 28, 28])


In [19]:
# Train Network
for epoch in range(num_epochs):
  for batch_idx,(data,targets) in enumerate(train_loader):
    # Get data to Cuda if possible
    #data=data.to(device=device).squeeze(1);
    data=data.to(device=device).squeeze(1);
    targets=targets.to(device=device);
    
    # Get data ro correct shape
    #data=data.reshape(data.shape[0],-1)
    #data=data.reshape(batch_size,sequence_length,input_size)


    # Forward
    scores=model(data);
    loss= criterion(scores,targets);

    # Backward
    optimizer.zero_grad();
    loss.backward();

    # Gradient Descent
    optimizer.step();

In [20]:
# Check accuracy on training & test to see how good our model

def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with \
              accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )
    # Set model back to train
    model.train()
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 58213 / 60000 with               accuracy 97.02
Checking accuracy on test data
Got 9693 / 10000 with               accuracy 96.93
